In [8]:
#dependencies 
#had to 
import pandas as pd 
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import config 
from coincallandwritepythonfile import refined_coin_list

In [6]:
refined_coin_list

,id,name,symbol,rank,File_Name,injest_datetime
1,1,Bitcoin,BTC,1,Coin_Ranking_Dim,04/15/2022 13:12:07
2,1027,Ethereum,ETH,2,Coin_Ranking_Dim,04/15/2022 13:12:07
3,825,Tether,USDT,3,Coin_Ranking_Dim,04/15/2022 13:12:07
4,1839,BNB,BNB,4,Coin_Ranking_Dim,04/15/2022 13:12:07
5,3408,USD Coin,USDC,5,Coin_Ranking_Dim,04/15/2022 13:12:07
...,...,...,...,...,...,...
97,4846,Kava,KAVA,97,Coin_Ranking_Dim,04/15/2022 13:12:07
98,8857,Anchor Protocol,ANC,98,Coin_Ranking_Dim,04/15/2022 13:12:07
99,2099,ICON,ICX,99,Coin_Ranking_Dim,04/15/2022 13:12:07
100,3801,BORA,BORA,100,Coin_Ranking_Dim,04/15/2022 13:12:07


In [11]:
#https://coinmarketcap.com/api/documentation/v1/#operation/getV2CryptocurrencyQuotesLatest
#now need to reference the coin ids and pull in data from the api accordinly 

config.coin_api_key

url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
parameters = {
    'id': [1]
    }
headers = {
'Accepts': 'application/json',
'X-CMC_PRO_API_KEY': config.coin_api_key
}

session = Session()
session.headers.update(headers)
response = session.get(url, params=parameters)
data = json.loads(response.text)


In [18]:
data['data']
cmc_df = pd.DataFrame.from_dict(data['data'], orient='index')

In [25]:
#need to explode this,
cmc_df['quote'][0]

{'USD': {'price': 40426.70598566949,
  'volume_24h': 21914610585.772533,
  'volume_change_24h': -11.5782,
  'percent_change_1h': 0.21933354,
  'percent_change_24h': 1.34169022,
  'percent_change_7d': -5.56060899,
  'percent_change_30d': -1.98471014,
  'percent_change_60d': -4.30594835,
  'percent_change_90d': -7.01802705,
  'market_cap': 768620549906.7964,
  'market_cap_dominance': 40.9128,
  'fully_diluted_market_cap': 848960825699.06,
  'last_updated': '2022-04-15T21:16:00.000Z'}}

In [26]:
768620549906.7964/ 40426.70598566949

19012693.0